# Download Searchanalytics Data
Herunterladen von Google Search Console Suchanalyse-Daten mit dem Python Package `searchconsole` [github link](https://github.com/joshcarty/google-searchconsole)

In [ ]:
import searchconsole

Verifizieren des API Zugriffs.

→ Wenn ihr das Notebook auf Binder.org ausführt, werden eure Credentials natürlich nicht gespeichert!

## Authentifizierung - Notebook auf mybinder.org
Wenn ihr das Notebook auf mybinder.org ausführt, öffnet die `.authenticate` Methode kein neues Fenster für die Authentifizierung. Ich gehe daher den Umweg über `pydata_google_auth`. 

In [ ]:
import json
import pydata_google_auth
from pydata_google_auth import cache
from googleapiclient.discovery import build
from searchconsole.account import Account

with open('client_secrets.json') as f:
    cs = json.load(f)

credentials = pydata_google_auth.get_user_credentials(scopes=['https://www.googleapis.com/auth/webmasters.readonly'],
                                                      client_id=cs['installed']['client_id'],
                                                      client_secret=cs['installed']['client_secret'],
                                                      credentials_cache=cache.REAUTH,
                                                      auth_local_webserver=False)

service = build('webmasters', 'v3', credentials=credentials)

account = Account(service, credentials)

## Authentifizierung - Lokales Notebook
Bei einem lokalen Notebook kann einfach wie für das Package `searchconsole` üblich fortgefahren werden.

In [ ]:
# account = searchconsole.authenticate(client_config='client_secrets.json')

## Daten abfragen

Liste der verifizierten Properties in dem Account

In [ ]:
account.webproperties

Webproperty Object mit dem wir den Query erstellen.

In [ ]:
webproperty = account['https://www.example.de/']

Query erstellen und ausführen.

In [ ]:
query = webproperty.query.range(start='2019-03-18', stop='2019-03-24') \
    .dimension('query')

In [ ]:
data = query.get()

So sieht das ganze als Pandas DataFrame aus. Hiermit kann man nun alles mögliche anstellen.

In [ ]:
df = data.to_dataframe()
df.head()

Eine Abfrage mit Filter.

In [ ]:
df = webproperty.query.range(start='2019-03-18', stop='2019-03-24') \
    .dimension('page','query') \
    .filter('page', 'https://www.example.de/', 'equals') \
    .get() \
    .to_dataframe()
df.head()

Export der Daten als CSV. Die Datei sollte heruntergeladen werden können. ✌️😉

In [ ]:
df.to_csv('gsc_searchanalytics_output.csv', header=True, index=False)

## Mit ipywidgets

In [ ]:
import pandas as pd
import ipywidgets as widgets
from tqdm import tqdm_notebook as tqdm
from IPython.display import FileLink
import time

def download_data(b):
    output.clear_output()
    with output:
        assert start.value is not None, 'Startdate not set'
        assert end.value is not None, 'Enddate not set'
        assert start.value <= end.value, 'Startdate after Enddate'

        export_to_csv_button.disabled = True
        b.description = '...in progress...'
        b.disabled = True
        try:
            global daily_data

            dates = [row.strftime('%Y-%m-%d') for row in pd.date_range(start=start.value, end=end.value, freq='D')]

            daily_data = []

            for date in tqdm(dates):
                data = webproperty_select.value.query.range(
                    start=date, stop=date
                ).dimension(
                    *dimension.value
                ).search_type(
                    searchtype.value
                ).get().to_dataframe()
                data['date'] = date
                daily_data.append(data)
                time.sleep(.3)

            daily_data = pd.concat(daily_data)
            daily_data['date'] = pd.to_datetime(daily_data['date'])
        except:
            raise
        else:
            export_to_csv_button.disabled = False
        finally:
            b.disabled = False
            b.description = 'Download from API'

def export_data(b):
    output.clear_output()
    with output:
        print('Exporting to CSV...')
        daily_data.to_csv(
            'gsc_searchanalytics_output.csv',
            header=True,
            index=False,
            sep=';',
            decimal=','
        )
        print('Done ✌️😉')
        display(FileLink('gsc_searchanalytics_output.csv'))

webproperty_select = widgets.Dropdown(
    options=[(row.url, row) for row in account.webproperties],
    description='Property',
    layout=widgets.Layout(flex='2 1 auto')
)

start = widgets.DatePicker(description='Startdate')
end = widgets.DatePicker(description='Enddate')

searchtype = widgets.Dropdown(
    options=sorted(['web','image','video']),
    value='web',
    description='Searchtype'
)

dimension = widgets.SelectMultiple(
    options=sorted(['country','device','page','query']),
    value=['query'],
    description='Dimensions'
)

download_data_button = widgets.Button(
    description='Download from API',
    tooltip='Start calling GSC API',
    layout=widgets.Layout(flex='2 1 auto')
)

export_to_csv_button = widgets.Button(
    description='Export to CSV',
    tooltip='Write GSC data to CSV file',
    disabled=True,
    layout=download_data_button.layout
)

download_data_button.on_click(download_data)
export_to_csv_button.on_click(export_data)

output = widgets.Output(
    layout=widgets.Layout(flex='2 1 auto')
)

widgets.VBox([
    widgets.HBox([webproperty_select,start,end]),
    widgets.HBox([dimension,searchtype]),
    widgets.HBox([download_data_button,
                  export_to_csv_button]),
    widgets.HBox([output])
])